In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
load_dotenv()


True

In [2]:
llm = ChatOpenAI(model='gpt-5-nano')

In [4]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}


In [5]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)


In [6]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'stock market'}, config=config1)

{'topic': 'stock market',
 'joke': 'Here are a few stock-market jokes. Pick your fave or I can tailor more:\n\n- Why did the stock market go to therapy? Too many emotional highs and lows.\n- I invested in a bakery to diversify my portfolio. Now my assets are in dough.\n- Why did the trader bring a ladder to the stock exchange? Because the stocks were going through the roof.',
 'explanation': 'Here are explanations for each joke, so you can see what makes them tick and why they’re funny.\n\n1) Why did the stock market go to therapy? Too many emotional highs and lows.\n- What’s funny: It uses a pun on “emotional highs and lows” (humans have mood swings; markets have price swings). The idea of a market “going to therapy” is playful because markets aren’t people, but treating market volatility like an emotional person makes the joke relatable.\n- The finance angle: “Highs and lows” refers to market peaks and troughs, i.e., volatility. Therapy is the unexpected, humorous twist that personif

In [7]:
workflow.get_state(config1)


StateSnapshot(values={'topic': 'stock market', 'joke': 'Here are a few stock-market jokes. Pick your fave or I can tailor more:\n\n- Why did the stock market go to therapy? Too many emotional highs and lows.\n- I invested in a bakery to diversify my portfolio. Now my assets are in dough.\n- Why did the trader bring a ladder to the stock exchange? Because the stocks were going through the roof.', 'explanation': 'Here are explanations for each joke, so you can see what makes them tick and why they’re funny.\n\n1) Why did the stock market go to therapy? Too many emotional highs and lows.\n- What’s funny: It uses a pun on “emotional highs and lows” (humans have mood swings; markets have price swings). The idea of a market “going to therapy” is playful because markets aren’t people, but treating market volatility like an emotional person makes the joke relatable.\n- The finance angle: “Highs and lows” refers to market peaks and troughs, i.e., volatility. Therapy is the unexpected, humorous 

In [8]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'fries'}, config=config2)

{'topic': 'fries',
 'joke': "Here's a quick fry joke: Why did the fry get promoted? It was outstanding in its field.",
 'explanation': 'Here’s why the joke lands:\n\n- Structure: It’s a setup-punchline joke. The setup asks about a promotion, which makes you think of a workplace win or ladder-climbing. The punchline flips that expectation.\n\n- The key pun: “outstanding” has two meanings:\n  - Literally: “standing out” or “standing in a field.”\n  - Figuratively: “exceptionally good” or “excellent.”\n\n- The word “field”:  \n  - In the literal sense, a field is where crops grow. Potatoes (which turn into fries) are grown in fields.  \n  - In the figurative sense, “in your field” means in your area of expertise.\n\n- The joke ties these together: a fry (a potato-based food) gets promoted because it’s “outstanding in its field.” It plays on being excellent in its domain while also literally being in a field (the field where potatoes grow). The humor comes from the double meaning and the s

In [9]:
list(workflow.get_state_history(config1))


[StateSnapshot(values={'topic': 'stock market', 'joke': 'Here are a few stock-market jokes. Pick your fave or I can tailor more:\n\n- Why did the stock market go to therapy? Too many emotional highs and lows.\n- I invested in a bakery to diversify my portfolio. Now my assets are in dough.\n- Why did the trader bring a ladder to the stock exchange? Because the stocks were going through the roof.', 'explanation': 'Here are explanations for each joke, so you can see what makes them tick and why they’re funny.\n\n1) Why did the stock market go to therapy? Too many emotional highs and lows.\n- What’s funny: It uses a pun on “emotional highs and lows” (humans have mood swings; markets have price swings). The idea of a market “going to therapy” is playful because markets aren’t people, but treating market volatility like an emotional person makes the joke relatable.\n- The finance angle: “Highs and lows” refers to market peaks and troughs, i.e., volatility. Therapy is the unexpected, humorous